In [4]:
! pip install numbers-parser

     |████████████████████████████████| 278 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 486 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 124 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 489 kB 8.9 MB/s eta 0:00:01
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4


In [38]:
import pandas as pd
import numpy as np

In [30]:
# Importing dataset
df = pd.read_csv('/Users/natashawyatt/Documents/Flatiron_school/capstone/wrap.csv')



In [39]:
# Taking a look at the first few rows...
df.head()

,Unnamed: 0,Date,Transaction Type,Num,Customer,Memo/Description,Qty,Sales Price,Amount,Balance
0,FireWrap Grip Kit - Light Blue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,03/23/2018,Sales Receipt,#1394,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,24.95
2,NaN,04/26/2018,Sales Receipt,#1477,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,49.90
3,NaN,04/27/2018,Sales Receipt,#1511,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,74.85
4,NaN,05/14/2018,Sales Receipt,#1617,NaN,FireWrap Grip Kit,1.0,34.95,34.95,109.80


In [40]:
# Checking the columns, Dtype, and number of rows..
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5801 entries, 0 to 5800
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        38 non-null     object 
 1   Date              5761 non-null   object 
 2   Transaction Type  5761 non-null   object 
 3   Num               5761 non-null   object 
 4   Customer          0 non-null      float64
 5   Memo/Description  5558 non-null   object 
 6   Qty               5780 non-null   float64
 7   Sales Price       5747 non-null   float64
 8   Amount            5780 non-null   object 
 9   Balance           5761 non-null   object 
dtypes: float64(3), object(7)
memory usage: 453.3+ KB


### Data Understanding
After just a quick look at the dataset we see there are 10 rows of which almost all of them are 'object' which means they are not in numerical form, the Non-Null Count also varies between column. The 'Unnamed : 0' column looks to be the product type and the 'Customer' column was cleared previously so no personal information would be shared. We will continue to look at each column individually in an effort to understand the data, and clean it in order for it to be used for modelling. For example, we need to understand what is the difference between 'Sales Price', 'Amount', and 'Balance'.
First lets look at the 'Unnamed: 0' column.
****
##### Unnamed: 0

In [43]:
df['Unnamed: 0'].unique()

array(['FireWrap Grip Kit - Light Blue', nan,
       'Total for FireWrap Grip Kit - Light Blue',
       'FireWrap Grip Kit - Pink', 'Total for FireWrap Grip Kit - Pink',
       'FireWrap® Grip Kit Black', 'Total for FireWrap® Grip Kit Black',
       'FireWrap® Grip Kit Blue', 'Total for FireWrap® Grip Kit Blue',
       'FireWrap® Grip Kit GLOW - Aqua',
       'Total for FireWrap® Grip Kit GLOW - Aqua',
       'FireWrap® Grip Kit GLOW - Green ( 927 )',
       'Total for FireWrap® Grip Kit GLOW - Green ( 927 )',
       'FireWrap® Grip Kit Orange', 'Total for FireWrap® Grip Kit Orange',
       'FireWrap® Grip Kit Red', 'Total for FireWrap® Grip Kit Red',
       'FireWrap® Grip Kit Yellow', 'Total for FireWrap® Grip Kit Yellow',
       'FWGK-GL-AQ', 'Total for FWGK-GL-AQ', 'FWGK-GL-GRN',
       'Total for FWGK-GL-GRN', 'FWGK-SC-BLK', 'Total for FWGK-SC-BLK',
       'FWGK-SC-BLU', 'Total for FWGK-SC-BLU', 'FWGK-SC-GRN',
       'Total for FWGK-SC-GRN', 'FWGK-SC-ORG', 'Total for FWGK-SC-ORG',

#### Product names
We see that there are a few issues here. First there is a product for each available color and the total sales for that product are in this column. They should be separated to make it easier to decipher in the pandas dataframe. Also I know that 'FWGK-SC-BLK' is the same product as 'FireWrap® Grip Kit Black', the reason for this is just the way they were logged into the company's system. I will attempt to combine the corresponding colors appropriately, but this does lead us to a question. Does the color of the Grip Kit have an impact on sales? The answer to this question is something that we will try to find in our EDA phase.

For now we will begin by renaming the column and then try to simplify the product names.

In [45]:
#Changing column name to Product Id:
df= df.rename(columns={"Unnamed: 0": "Product ID"})
df.head()

,Product ID,Date,Transaction Type,Num,Customer,Memo/Description,Qty,Sales Price,Amount,Balance
0,FireWrap Grip Kit - Light Blue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,03/23/2018,Sales Receipt,#1394,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,24.95
2,NaN,04/26/2018,Sales Receipt,#1477,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,49.90
3,NaN,04/27/2018,Sales Receipt,#1511,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,74.85
4,NaN,05/14/2018,Sales Receipt,#1617,NaN,FireWrap Grip Kit,1.0,34.95,34.95,109.80


In [46]:
# Replacing the 'FWGK-SC-BLK' with full product name for clarity..
df.replace({'FireWrap® Grip Kit Black':'FWGK-SC-BLK'}, inplace = True)


In [50]:
# Taking it one step more and making the product name easier to work with...
df['Product ID'].replace({'FireWrap® Grip Kit Black':'black_grip_kit'}, inplace=True)

*** 
This seems a little labor intensive to do for the 9 different colors, especially when we dont know if the colors will matter at the end of the day when it comes to the models. Lets create a function to make this a little easier.


In [ ]:
# WIP

def merge_rename_products(df, product_map, rename_to):
    # replace product Ids with new names
    df['Product'] = df['Product'].replace(product_map)
    # group by the new product names
    df = df.groupby(['Product']).sum()
    # rename the column
    df.rename(columns={'Product':rename_to}, inplace=True)
    return df


In [ ]:
# WIP

In [51]:
df.head()

,Product ID,Date,Transaction Type,Num,Customer,Memo/Description,Qty,Sales Price,Amount,Balance
0,FireWrap Grip Kit - Light Blue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,03/23/2018,Sales Receipt,#1394,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,24.95
2,NaN,04/26/2018,Sales Receipt,#1477,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,49.90
3,NaN,04/27/2018,Sales Receipt,#1511,NaN,FireWrap Grip Kit - Pre-Order,1.0,24.95,24.95,74.85
4,NaN,05/14/2018,Sales Receipt,#1617,NaN,FireWrap Grip Kit,1.0,34.95,34.95,109.80


In [34]:
df['Date'].nunique()

1467

In [35]:
df['Transaction Type'].unique()

array([nan, 'Sales Receipt', 'Invoice', 'Refund', 'Credit Memo'],
      dtype=object)

In [36]:
df['Num'].unique()

array([nan, '#1394', '#1477', ..., 'M-072022-001', 'M-090722-003',
       'M-092522-001'], dtype=object)

In [37]:
df['Memo/Description'].unique()

array([nan, 'FireWrap  Grip Kit - Pre-Order', 'FireWrap  Grip Kit',
       'P.O. Number: 19-0213-5 \nJason Oliva\n2040 N 51st St\nMilwaukee, WI 53208',
       'FireWrap Grip Kit', 'FireWrap Grip Kit - Light Blue',
       'FireWrap Grip Kit - Pink', 'P.O. Number: 19-0103-11',
       'P.O. Number: 19-0129-1 \nBradford McCutcheon\n10538 Le Mans Rd\nDallas, TX 75238',
       'P.O. Number: 19-0129-2\nSean Wilson\n5150 Hampshire Dr\nShelby Township, MI 48316',
       'FireWrap Grip-Kit für Halligan-Tool - Schwarz',
       'Custom Install - Halligan w/ red and Hammer all black.',
       'Thin Red Line',
       'P.O. Number: 19-0204-3 \nDayton Shireman\n537 SE Canterbury LN\nLees Summit, MO 64063',
       'Thin Red Line Edition', 'FireWrap® Grip Kit',
       'Matthew Sapp\n7234 Waters Edge Dr\nThe Colony TX 75056',
       'Dan Mallory\nFort Calhoun Fire-Rescue\n5543 Clearwater Dr\nFort Calhoun NE 68023',
       'Charles Cerone\n282 3rd St \nSaddle Brook NJ 07663',
       'Alex Tull\n417 E Libe